# Carregando Dados Experimentais

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.signal import resample
import itertools


## Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization, Conv1D, SpatialDropout1D, Dropout, MaxPooling1D, BatchNormalization, Dense, Flatten, LSTM, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2, L1L2
from tensorflow.keras.constraints import unit_norm

# **Matriz de confusão**
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score


# **Classification Report**
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# **Curva ROC**
from scipy import interp
from sklearn.metrics import roc_curve, auc

In [2]:
def extract_data(path, files, skip_rows):
    df = pd.concat((pd.read_csv(path + files[f] + '.csv', skiprows=[i for i in range(1, skip_rows)]).assign(filename=f) for f in range(len(files))), ignore_index=True)
    df.columns.values[:] = ['Va', 'Vb', 'Vc', 'Ia', 'Ib', 'Ic', 'label']

    return df

In [3]:
files =  ['struct_rs_R1.mat_torque05', 'struct_rs_R1.mat_torque10', 'struct_rs_R1.mat_torque15', 'struct_rs_R1.mat_torque20', 'struct_rs_R1.mat_torque25','struct_rs_R1.mat_torque30', 'struct_rs_R1.mat_torque35', 'struct_rs_R1.mat_torque40', 
          'struct_r1b_R1.mat_torque05', 'struct_r1b_R1.mat_torque10', 'struct_r1b_R1.mat_torque15', 'struct_r1b_R1.mat_torque20', 'struct_r1b_R1.mat_torque25','struct_r1b_R1.mat_torque30', 'struct_r1b_R1.mat_torque35', 'struct_r1b_R1.mat_torque40',
          'struct_r2b_R1.mat_torque05', 'struct_r2b_R1.mat_torque10', 'struct_r2b_R1.mat_torque15', 'struct_r2b_R1.mat_torque20', 'struct_r2b_R1.mat_torque25','struct_r2b_R1.mat_torque30', 'struct_r2b_R1.mat_torque35', 'struct_r2b_R1.mat_torque40',
          'struct_r3b_R1.mat_torque05', 'struct_r3b_R1.mat_torque10', 'struct_r3b_R1.mat_torque15', 'struct_r3b_R1.mat_torque20', 'struct_r3b_R1.mat_torque25','struct_r3b_R1.mat_torque30', 'struct_r3b_R1.mat_torque35', 'struct_r3b_R1.mat_torque40',
          'struct_r4b_R1.mat_torque05', 'struct_r4b_R1.mat_torque10', 'struct_r4b_R1.mat_torque15', 'struct_r4b_R1.mat_torque20', 'struct_r4b_R1.mat_torque25','struct_r4b_R1.mat_torque30', 'struct_r4b_R1.mat_torque35', 'struct_r4b_R1.mat_torque40']
path = 'C:/Users/vinic/Desktop/LXL/USP/Dataset_USP_CSV/'

In [4]:
df = extract_data(path=path, files=files, skip_rows=194657)

In [5]:
# Resample 833 pontos por ciclo para 128

df0 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 3]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df0 = np.concatenate([resample(df0[i], 128).reshape(1, -1) for i in range(df0.shape[0])], axis=0)

df1 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 4]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df1 = np.concatenate([resample(df1[i], 128).reshape(1, -1) for i in range(df1.shape[0])], axis=0)

df2 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 5]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df2 = np.concatenate([resample(df2[i], 128).reshape(1, -1) for i in range(df2.shape[0])], axis=0)

df3 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 0]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df3 = np.concatenate([resample(df3[i], 128).reshape(1, -1) for i in range(df0.shape[0])], axis=0)

df4 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 1]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df4 = np.concatenate([resample(df4[i], 128).reshape(1, -1) for i in range(df1.shape[0])], axis=0)

df5 = np.array(np.concatenate([np.array_split(np.array(df.iloc[:, 2]).reshape(968*40, 833), 40)[i] for i in range(40)], axis=0))
df5 = np.concatenate([resample(df5[i], 128).reshape(1, -1) for i in range(df2.shape[0])], axis=0)


# Concatena todos os valores extraidos em uma unica matriz
datasetCon = np.concatenate((df0, df1, df2, df3, df4, df5), axis=1)

datasetCon.shape

(38720, 768)

In [6]:
# Adiciona os Labels
labels = np.concatenate([np.full((968, 1), i, dtype=int) for i in range(40)], axis=0)

datasetCon = np.concatenate((datasetCon, labels),axis=1)

# Treino, Teste e Validação

In [7]:
""" Aplicando one-hot-encoding, as categorias se transformam em colunas (variáveis) onde o 
número 1 representa o valor afirmativo e o 0 negativo. 
https://arthurlambletvaz.medium.com/one-hot-encoding-o-que-%C3%A9-cd2e8d302ae0 """

"""60% - Treino, 20% - Validação, 20% - Teste"""

# Coleta os dados das entradas datasetCon (ficando de fora os dados do One-hot-encoder)
x_datasetCon = datasetCon[:, :-1]

# Coleta os dados das saídas
y_datasetCon = datasetCon[:, -1]

# Separa os dados de entrada e saida para treino e testes
x_train,x_temp,y_train,y_temp = train_test_split(x_datasetCon,y_datasetCon, test_size=0.3, random_state=42, shuffle=True, stratify=y_datasetCon)
x_test,x_validation,y_test,y_validation = train_test_split(x_temp,y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)



"""Tamanho do teste e Validação (X, Y)"""

# Treino
print(x_train.shape, y_train.shape)

# Validação
print(x_validation.shape, y_validation.shape)

# Test
print(x_test.shape, y_test.shape)

"""Normalização"""

scaler  = MinMaxScaler(feature_range=(0,1)) # Tangente Hiperbólica ou outro análoga

x_train       = scaler.fit_transform(x_train)
x_validation  = scaler.fit_transform(x_validation)
x_test        = scaler.fit_transform(x_test)

(27104, 768) (27104,)
(5808, 768) (5808,)
(5808, 768) (5808,)


# CNN

In [9]:
# Começo da CNN
inputs = Input((x_train.shape[1],1))
    
# Primeira camada convolucional
Conv = Conv1D(filters=32, 
            kernel_size=6, 
            strides=1, 
            activation='relu', 
            padding='same', 
            kernel_regularizer=L1L2(l1=1e-4, l2=1e-3),
            bias_regularizer=L1L2(l1=1e-4, l2=1e-3))(inputs)

Conv = keras.layers.SpatialDropout1D(0.05)(Conv)
Conv = MaxPooling1D(pool_size=3, strides=None, padding='same')(Conv)
Conv = BatchNormalization()(Conv)
    
# Segunda camada convolucional
Conv = Conv1D(filters=32, 
            kernel_size=6, 
            strides=1, 
            activation='relu', 
            padding='same', 
            kernel_regularizer=L1L2(l1=1e-4, l2=1e-3),
            bias_regularizer=L1L2(l1=1e-4, l2=1e-3))(Conv)

Conv = keras.layers.SpatialDropout1D(0.05)(Conv)
Conv = MaxPooling1D(pool_size=3, strides=None, padding='same')(Conv)
Conv = BatchNormalization()(Conv)

# Terceira camada convolucional
Conv = Conv1D(filters=32, 
            kernel_size=6, 
            strides=1, 
            activation='relu', 
            padding='same', 
            kernel_regularizer=L1L2(l1=1e-4, l2=1e-3),
            bias_regularizer=L1L2(l1=1e-4, l2=1e-3))(Conv)

Conv = keras.layers.SpatialDropout1D(0.05)(Conv)
Conv = MaxPooling1D(pool_size=3, strides=None, padding='same')(Conv)
Conv = BatchNormalization()(Conv)
                    
# Flatten
dense = Flatten()(Conv)

# Primeira camada densa
dense = Dense(units=512, 
            activation='relu',
            kernel_regularizer=L1L2(l1=1e-4, l2=1e-3),
            bias_regularizer=L1L2(l1=1e-4, l2=1e-3),
            activity_regularizer=L1L2(l1=1e-4, l2=1e-3))(dense)
dense = keras.layers.Dropout(0.05)(dense)


# Segunda camada densa
dense = Dense(units=64, 
            activation='relu',
            kernel_regularizer=L1L2(l1=1e-4, l2=1e-3),
            bias_regularizer=L1L2(l1=1e-4, l2=1e-3),
            activity_regularizer=L1L2(l1=1e-4, l2=1e-3))(dense)
dense = keras.layers.Dropout(0.05)(dense)

# Camda de saída
dense = Dense(len(pd.DataFrame(y_test).value_counts()), activation='softmax')(dense)
    


# instantiate and compile model
model = Model(inputs, dense)
opt = Adam(learning_rate = 0.000262)


model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['acc'])

model.summary()



""" Callbacks """

# Check Point
checkpoint_cb = ModelCheckpoint('CNN_Optuna.h5', save_best_only=True, monitor='val_acc', mode='auto')

# Early Stopping
earlystopping = EarlyStopping(monitor="val_acc", min_delta=1e-3, patience=50, mode='auto', restore_best_weights=True)

# Reduce LR
reduceLR = ReduceLROnPlateau(monitor="val_acc", factor=0.8, patience=20, mode="auto", min_delta=1e-3, cooldown=0, min_lr=0)

        
# fit the model
history = model.fit(x_train, y_train, epochs=300,
                    validation_data=(x_validation, y_validation),
                    batch_size=64, 
                    callbacks=[[checkpoint_cb, 
                                earlystopping, 
                                reduceLR]])
            
# Evaluate the model accuracy on the test set.
score = model.evaluate(x_test, y_test)
            

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 768, 32)           224       
                                                                 
 spatial_dropout1d (SpatialD  (None, 768, 32)          0         
 ropout1D)                                                       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 256, 32)          0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 256, 32)          128       
 ormalization)                                                   
                                                             

In [10]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter( y=history.history['val_loss'], name="val_loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=history.history['loss'], name="loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=history.history['val_acc'], name="val accuracy"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter( y=history.history['acc'], name="accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss/Accuracy of CNN Model"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

# Set y-axes titles
fig.update_yaxes(title_text="Loss", secondary_y=False)
fig.update_yaxes(title_text="Accuracy", secondary_y=True)

fig.show()

In [11]:
# Evaluate the model accuracy on the test set.
score = model.evaluate(x_test, y_test, verbose=1)

print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

saida_rede = pd.DataFrame(model.predict(x_test)).round(decimals = 2)
predito   = saida_rede.idxmax(axis=1)
avaliado  = pd.DataFrame(y_test).iloc[:, 0]

score = accuracy_score(avaliado, predito)
print('Acurácia Teste')
print(score)


print("f1:",f1_score(y_test, predito,average='micro'))

print ("precision:",precision_score(y_test, predito,average='macro'))

print ("recall:",recall_score(y_test, predito,average='macro'))

print(classification_report(y_test, predito))

182/182 [==============================] - 2s 9ms/step - loss: 0.1870 - acc: 0.9745
acc: 97.45%
182/182 [==============================] - 2s 8ms/step
Acurácia Teste
0.9743457300275482
f1: 0.9743457300275482
precision: 0.9772378313477855
recall: 0.9743823807274445
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       145
         1.0       1.00      1.00      1.00       145
         2.0       1.00      1.00      1.00       146
         3.0       1.00      0.99      1.00       146
         4.0       0.99      1.00      1.00       145
         5.0       1.00      1.00      1.00       145
         6.0       1.00      1.00      1.00       145
         7.0       1.00      1.00      1.00       145
         8.0       1.00      1.00      1.00       145
         9.0       1.00      1.00      1.00       145
        10.0       0.99      1.00      0.99       145
        11.0       1.00      0.94      0.97       145
        12.0       0.98      0.9

# Matriz Confusao Teste

In [12]:
cm = confusion_matrix(avaliado, predito)
# Normalise
cmn = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
fig, ax = plt.subplots(figsize=(20,20), dpi=600)

sns.heatmap(cmn, 
            annot=True, 
            cmap="Blues",
            cbar = False)


plt.title('Matriz de Confusão', fontsize=18, fontweight="bold")
plt.ylabel('Classe Real', fontsize=16, fontweight="bold")
plt.xlabel('Classe Predita', fontsize=16, fontweight="bold")
#plt.savefig("matriz_confusao_experimental", dpi=600)
plt.show(block=False)

# Curva ROC Teste

In [14]:
# **Curva ROC**

#creating instance of one-hot-encoder
encoder = OneHotEncoder()

#perform one-hot encoding on 'team' column
y_test_hot = encoder.fit_transform(y_test.reshape(-1, 1)).toarray()
predito_hot = encoder.fit_transform(np.array(predito).reshape(-1,1)).toarray()
#predito_hot = np.insert(predito_hot, 37, np.zeros((5808, 1)).ravel(), axis=1)

print()
print(y_test_hot.shape)
print(predito_hot.shape)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(40):
    fpr[i], tpr[i], _ = roc_curve(y_test_hot[:,i], predito_hot[:,i], )
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_hot.ravel(), predito_hot.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])


fig = px.area(
    x=fpr[16], y=tpr[16],
    title=f'Curva ROC - Classe 16',
    labels=dict(x='Taxa de Falso Positivo', y='Taxa de Verdadeiro Positivo'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)

fig.add_annotation(x=0.4, y=0.8, text=f'Area={auc(fpr["micro"], tpr["micro"]):.4f}', showarrow=False)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')
fig.show()


(5808, 40)
(5808, 40)
